In [2]:
import numpy as np
import pydicom as pyd
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 


In [3]:
data_path='./Lung-DICOM'


In [6]:
# this function is fixing the missing values (if any) in the Slice Thickness Metadata
def load_scan(path):#We can also do it for the other code, do it later.
    # replace it with os.walk
    slices=[pyd.dcmread(path+'/'+s) for s in os.listdir(path)]#List comprehension
    list.sort(key=,dtype)
    #Sort the list by Instance number, instance is a unique key in each of the DICOM file
    slices.sort(key=lambda x: int(x.InstanceNumber)) 
    try:#Slice thickness is along z axis, but we dont know actual coordinates because of reference issues, so 
        #we define slice_thickness as difference.
        silce_thickness=np.abs(slices[0].ImagePositionPatient[2]-slices[1].ImagePositionPatient[2])
    except:
        # Slice location and image position [2] are same, this error handler ensures that if there is a missing value in image position, it takes slice location
    
        slice_thickness=np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:#For every slice it will set the attribute to the value of above array
        s.SliceThickness=slice_thickness
    return slices

    
def get_pixels_hu(scans):
    image=np.stack(list(ds.pixel_array for ds in scans))#We have stacked all the arrays(pixel arrays) as one. 
    image=image.astype(np.int16)#Image is the' combined effect' of slices' pixels data
    intercept=scans[0].RescaleIntercept# These two attributes are already present in ds(ResInter and Slope)
    slope=scans[0].RescaleSlope
    image[image == -2000] = 0 #
    if slope !=1:#slope is assumed to be one generally
        image = slope * image.astype(np.float64)
        image=image.astype(np.int16)
    image=image+np.int16(intercept) #Formula for hu conversion: slope*image+intercept
    return np.array(image, dtype=np.int16)

       
    
    

In [7]:
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)# for the second function patient is scans
# It should be noted that imgs contain pixel data and array used.


In [10]:
print(patient)

[(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.40744.29.309093436465192420112014607036573091300
(0008, 0020) Study Date                          DA: '20180212'
(0008, 0021) Series Date                         DA: '20180212'
(0008, 0022) Acquisition Date                    DA: '20180212'
(0008, 0023) Content Date                        DA: '20180212'
(0008, 0030) Study Time                          TM: '215029'
(0008, 0031) Series Time                         TM: '215029'
(0008, 0032) Acquisition Time                    TM: '215029'
(0008, 0033) Content Time                        TM: '215029'
(0008, 0050) Accession Number                    SH: '2819497684894126'
(0008, 0060) Modality                            CS: 'CT'
(0008, 0070) M

In [8]:
print(imgs)#This has loaded all the files within

[[[-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  ...
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]]

 [[-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  ...
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]]

 [[-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  ...
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]]

 ...

 [[-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  [-1024 -1024 -1024 ... -1024 -1024 -1024]
  ...
  [-1024 -1024 -1024 ... -1024 -1024 -10

In [ ]:
np.save(output_path+)